In [135]:
import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from annoy import AnnoyIndex
from datetime import datetime

In [3]:
def recurse_playlist(id):
    r = sp.playlist(id)
    playlists.add(r)
    if 'tracks' in r:
        if 'next' in r['tracks']:
            if r['tracks']['next'] is not None:
                recurse_playlist(str.replace(r['tracks']['next'], 'https://api.spotify.com/v1/playlists/', ''))
    elif 'next' in r:
        if r['next'] is not None:
            recurse_playlist(str.replace(r['next'], 'https://api.spotify.com/v1/playlists/', ''))
    else:
        pass

In [4]:
def get_metadata(ids):
    metadata = list()
    # 50 limit on api call    
    for i in range(0,len(ids),50):
        meta = sp.tracks(ids[i:i+50])
        for data in meta['tracks']:
            artists = list()
            if data is not None:
                for artist in data['artists']:
                    artists.append(artist['id'])
                metadata.append({"song_id":data['id'],  
                                 "artists":artists, 
                                 "song_name":data['name'], 
                                 "album_id":data['album']['id'],
                                 "album_name":data['album']['name'],
                                 "popularity":data['popularity']})
    return pd.DataFrame(metadata)

In [5]:
def get_features(ids):
    features = []
    # 50 limit on api call
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        for track in audio_features:
            if track is not None:
                features.append(track)
    
    return pd.DataFrame(features)

In [6]:
def recurse_saved_songs(offset=0):
    r = sp.current_user_saved_tracks(20, offset)
    for i in r['items']:
        songs.append({"id":i['track']['id'], "added_at":i['added_at']})
    if r['next']:
        offset+=20
        recurse_saved_songs(offset)

In [7]:
cid ="0deb154cdea34cfa9c50fc76938403b9" 
secret = "6aa3c3ee390d4421bdc6a860cf33c686"
username = "Ben"

In [9]:
credentials = SpotifyClientCredentials(
        client_id=cid,
        client_secret=secret)

In [10]:
sp = spotipy.Spotify(auth_manager=credentials)

In [48]:
user_ids = [
    'kirstendodo',
    '1260745480',
    '1255526461',
    'athenajiang',
    '1239664093',
    '1268639225',
    '22eja5lr4xyqrmdgb4jox74pi',
    'russellkim98',
    '1213055384',
    '12563360',
    '1218288200',
    '22vxpm5y7bws42rgecwwdtqzq',
    '2lb6yucjb2mmlbpqvv7f6tpa',
    'hahowie',
    '1233302457',
    'clairehuangg',
    'gmfrlife',
    'karawho',
    '1210134132',
    'gracehuang22',
    'sangeethaisrofl',
    '124028238',
    '1228739853',
    '1249053478',
    '1249402231',
    '1261634901',
    'jasonliao',
    '12101413836',
    '227lrr7ufyoef3thjhnzjcbhi',
    '1263186670',
    '126549512',
    '22tbkcbk3npj7dmozb5xcvegq',
    '1258256267',
    'zenbhang',
    '1249768840',
    '1251291991',
    '1293249673',
    '12184871363',
    '1225107231',
    '21jm5sg2qelg57la434krzpjy',
    '22n5d3jkyzx6ucmdh6stknedi'
]

In [49]:
playlist_ids = set()
for ui in user_ids:
    try:
        r = sp.user_playlists(ui, limit=50)
        for i in r['items']:
            playlist_ids.add(i['id'])
    except:
        print(ui, 'error')

HTTP Error for GET to https://api.spotify.com/v1/users/2lb6yucjb2mmlbpqvv7f6tpa/playlists with Params: {'limit': 50, 'offset': 0} returned 404 due to Not found.


2lb6yucjb2mmlbpqvv7f6tpa error


In [13]:
playlists = set()
for i in playlist_ids:
    recurse_playlist(str.replace(i, 'https://open.spotify.com/playlist/', ''))

In [14]:
ids = set()
for p in playlists:  
    if 'tracks' in p:
        for item in p['tracks']['items']:
            try:
                if item['track']['id'] is not None:
                    ids.add(item['track']['id'])
            except:
                pass
    elif 'items' in p:
        for item in p['items']:
            try:
                if item['track']['id'] is not None:
                    ids.add(item['track']['id'])
            except:
                pass

In [15]:
df_metadata = get_metadata(ids)

In [17]:
df_features = get_features(ids)
# We probably just want to build a db with all of these song + features

In [176]:
len(df_metadata)

57502

In [177]:
df_metadata.loc[[5773, 2176, 54482, 35915, 40399, 20884, 9975, 20049, 21284, 35395, 17865, 18383, 7704, 7742, 20480, 26160, 9222, 8400, 44129, 35589, 7778, 18590, 3200, 25901, 55937],:]

,song_id,artists,song_name,album_id,album_name,popularity
5773,73grS4MZ4rHwa7CJOihO73,[1RyvyyTE3xzB2ZywiAwp0i],Solo,1P8NvRvykmDrKyfglMerMv,HNDRXX,20
2176,7qRqmkHtKgk9B8FTGp9YjL,"[0M7GyeyRi2fG8c1LdP4jhi, 5Pb27ujIyYb33zBqVysBkj]",No Time - RÜFÜS DU SOL Remix,4uYah5RDbWZkcytZ5OwdEy,No Time (RÜFÜS DU SOL Remix),47
54482,5XU99uJiaZIlfy1ID23veH,"[3IG3Ub4ra8AuSxCFDVkVco, 6d1k1QqEg5v7gVCJmbilHG]",Should Have Seen It Coming,6yVQrYmOxFjvS4875yrfGa,Should Have Seen It Coming,52
35915,1z2jZ0HMcxPqUq8NoFGoxp,[02uPe16VFxPaiueQsPEDkE],Red-Eye,1pFrCdJfefM5UgijeacUpB,Into The Blue Again,0
40399,2tGowgO4B6qFluKZR1zSAC,[4kubsO16bEfCADaVUyoYb6],Waste Yr Time,3Qrd5i1iAguhCSZtCQOU42,Comfort,17
20884,2Th9BGKvfZG8bKQSACitwG,"[7wg1qvie3KqDNQbAkTdbX0, 0pCNk4D3E2xtszsm6hMsWr]",It's Strange (feat. K.Flay),14qZ1kKG6UoQnBupQTtYRq,It's Strange (feat. K.Flay),45
9975,4KGdJZCwVT2FYQOA3E3ohB,[19KYorPPDJewrAXMeoZccy],Ask For Absolution - Original Mix,5MQcSnYKtU5BeO8FmO2Zmr,Skull Crushing Techniques (1),19
20049,3Fd2frAOImvpvDO8gfYIEZ,[3FBxzgzGPs9XUfSYZgG1vX],The Raw,5XfxMXUOkXB99ENYEkgg0u,The Raw,0
21284,2kpziA4mHuM7Na6La4upg1,"[0lzV2CiahHRiGd6qpADtPS, 2IjiuEObrCKyZvSb8xLxG9]",Coming Over - filous Remix,24zMrWtj8r3QmczFyWhrPM,Coming Over (filous Remix),9
35395,6OcRxChYGUuG1We308oMLz,[2ABBMkcUeM9hdpimo86mo6],For Sarah,3V0mDywNFRF9zqzG8L8IPr,U,15


# User Features

In [129]:
scope = ['user-library-read', "playlist-modify-public"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = "0deb154cdea34cfa9c50fc76938403b9",
                                               client_secret = secret,
                                               redirect_uri= 'http://127.0.0.1:5000/spotify/callback',
                                               scope=scope,
                                              open_browser=True))

In [ ]:
# There's this 
# GET https://api.spotify.com/v1/me/top/{type}


In [ ]:
# if you authorize this app, we should take your profile URL and 

In [83]:
df_metadata.loc[26160]

song_id         375iBNYeivMA2p63yAObz1
artists       [3534yWWzmxx8NbKVoNolsK]
song_name         Redline - Radio Edit
album_id        0rbu1eSVOn2VQAm9AA7WLl
album_name                     Redline
popularity                           0
Name: 26160, dtype: object

In [82]:
df_features.loc[26160]

danceability                                                    0.724
energy                                                          0.848
key                                                                 7
loudness                                                       -4.638
mode                                                                1
speechiness                                                     0.143
acousticness                                                  0.00311
instrumentalness                                               0.0736
liveness                                                        0.175
valence                                                        0.0386
tempo                                                         127.949
type                                                   audio_features
id                                             1PEl4zj4XIyIwzULRXlhDS
uri                              spotify:track:1PEl4zj4XIyIwzULRXlhDS
track_href          

In [85]:
df_metadata[df_metadata['song_id']=='5fWG4JDot8nHCte7gUdMUS']

,song_id,artists,song_name,album_id,album_name,popularity


In [19]:
songs = list()
recurse_saved_songs()

In [20]:
df_your_songs = pd.DataFrame(songs)

In [174]:
df_your_songs.head()

,id,added_at
0,5fWG4JDot8nHCte7gUdMUS,2021-10-09T22:39:45Z
1,4piGD6lbuj57rxausi0brN,2021-10-06T23:12:48Z
2,5RQmbhnkrt6llPrxqRsym4,2021-10-03T12:18:34Z
3,75sMhi81W5Tw2AGlOGATuy,2021-09-28T03:56:05Z
4,0OOT6JmtkcYidsuQ9Fh54i,2021-09-21T16:30:28Z


In [162]:
df_nearest_neighbors_indices = get_nn(index, df_your_songs.head()['id'])

In [163]:
df_nearest_neighbors_indices

,song_index,distance
0,2176,0.005079
1,3200,0.002026
2,5773,0.005177
3,7704,0.002963
4,7742,0.002963
5,7778,0.002334
6,8400,0.002429
7,9222,0.002429
8,9975,0.004034
9,17865,0.003181


In [164]:
yoo = apply_filtering_criteria(df_nearest_neighbors_indices)

In [173]:
songs = df_metadata.iloc[list(yoo['song_index']), :]['song_id']

In [166]:
songs.head()

,song_id,artists,song_name,album_id,album_name,popularity
5773,73grS4MZ4rHwa7CJOihO73,[1RyvyyTE3xzB2ZywiAwp0i],Solo,1P8NvRvykmDrKyfglMerMv,HNDRXX,20
2176,7qRqmkHtKgk9B8FTGp9YjL,"[0M7GyeyRi2fG8c1LdP4jhi, 5Pb27ujIyYb33zBqVysBkj]",No Time - RÜFÜS DU SOL Remix,4uYah5RDbWZkcytZ5OwdEy,No Time (RÜFÜS DU SOL Remix),47
54482,5XU99uJiaZIlfy1ID23veH,"[3IG3Ub4ra8AuSxCFDVkVco, 6d1k1QqEg5v7gVCJmbilHG]",Should Have Seen It Coming,6yVQrYmOxFjvS4875yrfGa,Should Have Seen It Coming,52
35915,1z2jZ0HMcxPqUq8NoFGoxp,[02uPe16VFxPaiueQsPEDkE],Red-Eye,1pFrCdJfefM5UgijeacUpB,Into The Blue Again,0
40399,2tGowgO4B6qFluKZR1zSAC,[4kubsO16bEfCADaVUyoYb6],Waste Yr Time,3Qrd5i1iAguhCSZtCQOU42,Comfort,17


In [154]:
yoo.dtypes

song_id    int64
dtype: object

In [59]:
len(df_features)

57500

In [25]:
# index can just be the integer : song id mapping LMFAO
df = df_features.reset_index()

In [30]:
test = np.array(df_features[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']])

In [70]:
index = build_annoy_index(test)
index.save('data/test.ann')

True

In [63]:
def build_annoy_index(array, trees=10):
    # vector size is feature length, idk wtf angular does
    t = AnnoyIndex(11, 'angular')
    for i in range(len(array)):
        t.add_item(i, test[i])
    # once this is built, you can't add items to it. Should we build it somewhere else?
    t.build(trees)
    
    return t

In [ ]:
# select song ids that are not already in 
# worry about this later
# right now all we need to do is get the feature vectors for the songs you've loaded and check for nearest

In [1]:

# annoy is basically
# [1: [feature1, feature2, feature3]]
# it only supporst integers as identifiers for items, so will have to map
# t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
#     t.add_item(i, v)
# t.build(10) # 10 trees
# print(t.get_nns_by_item(0, 1000)) # will find the 1000 nearest neighbors

In [ ]:
def get_songs_for_nn():
# Apply some sampling criteria to get a bunch of songs based on the user's last X days of activity (sampling? most recent)
    
    return pass

In [161]:
def get_nn(t, songs):
    nearest_neighbors = list()
    song_features = np.array(get_features(songs)[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']])
    for s in song_features:
        nearest_neighbors.append(t.get_nns_by_vector(s, 5, search_k=-1, include_distances=True))
        
    df_nearest_neighbors_indices = pd.DataFrame()
    for nn in nearest_neighbors:
        # unpacks tuple and appends to df
        df_nearest_neighbors_indices = df_nearest_neighbors_indices.append(
            pd.DataFrame.from_dict(dict(zip(nn[0], nn[1])), 
                                   orient='index'))
    df_nearest_neighbors_indices = (
        df_nearest_neighbors_indices
        .reset_index()
        .groupby('index', as_index=False)
        .min()
        .rename(columns={"index":"song_index", 0:"distance"})
    )    
    # for each song in songs, we get a few close songs. Return all of them?
    return df_nearest_neighbors_indices

In [150]:
def apply_filtering_criteria(songs_df):
    # stick a bunch of rules on there to get us to 30 songs
    # right now let's just take top 30
    return songs_df.sort_values(by='distance', ascending=False).head(30)[['song_index']]
    

In [168]:
def create_playlist(songs):
    user_id = sp.me()['id']
    timestampStr = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
    sp.user_playlist_create(user_id, f'Customized Discover Weekly - {timestampStr}')
    # We don't get the playlist id when we create it, so we take the user's first 50 playlists, then find the name we gave it
    playlists = sp.user_playlists(sp.me()['id'])['items']
    playlist_names = {}
    for i in playlists:
        playlist_names[i['name']] = i['id']
    playlist_id = playlist_names[f'Customized Discover Weekly - {timestampStr}']
    sp.playlist_add_items(playlist_id, songs)

In [170]:
len(songs)

25

In [169]:
create_playlist(list(songs['song_id']))

In [137]:
playlists = sp.user_playlists(sp.me()['id'])['items']

In [140]:
playlist_names = {}
for i in playlists:
    playlist_names[i['name']] = i['id']

In [180]:
np.random.randint(100,101 + 100,size=(100, 1))/100

array([[1.14],
       [1.16],
       [1.68],
       [1.51],
       [1.44],
       [1.01],
       [1.47],
       [1.75],
       [1.76],
       [1.28],
       [1.36],
       [1.78],
       [1.08],
       [1.47],
       [1.05],
       [1.04],
       [1.1 ],
       [1.87],
       [1.09],
       [1.13],
       [1.56],
       [1.11],
       [1.66],
       [1.35],
       [1.56],
       [1.29],
       [1.38],
       [1.9 ],
       [1.42],
       [1.59],
       [1.56],
       [1.84],
       [1.54],
       [1.92],
       [1.35],
       [1.28],
       [1.8 ],
       [1.56],
       [1.84],
       [1.71],
       [1.46],
       [1.64],
       [1.42],
       [1.95],
       [1.38],
       [1.93],
       [1.36],
       [1.75],
       [1.29],
       [1.32],
       [1.74],
       [1.11],
       [1.52],
       [1.2 ],
       [1.54],
       [1.49],
       [1.97],
       [1.14],
       [1.7 ],
       [2.  ],
       [1.7 ],
       [1.87],
       [1.46],
       [1.32],
       [1.49],
       [1.08],
       [1.

In [182]:
if type(3) == int:
    print('i')

i
